In [150]:
import cv2
import rasterio
from rasterio.windows import Window
import geopandas as gpd
from PIL import Image
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from shapely import Polygon
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
from mmseg.apis import MMSegInferencer
import shapely
import torch.nn.functional as F

In [151]:
def to_numpy2(transform):
    return np.array([transform.a,
                     transform.b,
                     transform.c,
                     transform.d,
                     transform.e,
                     transform.f, 0, 0, 1], dtype='float64').reshape((3, 3))


def xy_np(transform, rows, cols, min_x, min_y, offset='center'):
    if isinstance(rows, int) and isinstance(cols, int):
        pts = np.array([[rows+min_y, cols+min_x, 1]]).T
    else:
        assert len(rows) == len(cols)
        pts = np.ones((3, len(rows)), dtype=int)
        pts[0] = rows + min_y
        pts[1] = cols + min_x
    if offset == 'center':
        coff, roff = (0.5, 0.5)
    elif offset == 'ul':
        coff, roff = (0, 0)
    elif offset == 'ur':
        coff, roff = (1, 0)
    elif offset == 'll':
        coff, roff = (0, 1)
    elif offset == 'lr':
        coff, roff = (1, 1)
    else:
        raise ValueError("Invalid offset")
    _transnp = to_numpy2(transform)
    _translt = to_numpy2(transform.translation(coff, roff))
    locs = _transnp @ _translt @ pts
    lat, lon = locs[0].tolist(), locs[1].tolist()
    coords = [(lat[i], lon[i]) for i in range(len(lat))]
    return coords

def polygons_from_binary_image(img, transform, crs, min_x=0, min_y=0, min_area=5):

    assert isinstance(img, np.ndarray), 'img deve ser um numpy array'

    unique_values = np.unique(img)

    new_geo_data_frame = {"geometry": [], 'CLASSE': []}

    for cat in unique_values:
        if cat == 0:
            continue

        img_ = img.copy()
        img_[img_ != cat] = 0
        img_[img_ != 0] = 1

        contours, hierarchy = cv2.findContours(img_, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        categ = cat

        interiors = [xy_np(transform, contour[:, 0, 0], contour[:, 0, 1], min_x, min_y)
                     for c, contour in enumerate(contours) if hierarchy[0][c][3] != -1]
        index = [
            hierarchy[0][c][3]
            for c, contour in enumerate(contours)
            if hierarchy[0][c][3] != -1
        ]

        for c, contour in tqdm(enumerate(contours)):
            if hierarchy[0][c][3] == -1:
                if cv2.contourArea(contour) < min_area:
                    continue
                exterior = xy_np(
                    transform, contour[:, 0, 0], contour[:, 0, 1], min_x, min_y)
                interior = [hole for h, hole in enumerate(
                    interiors) if index[h] == c]
                if len(exterior) <= 3:
                    continue
                poly = shapely.geometry.polygon.Polygon(exterior, interior)
                new_geo_data_frame["geometry"].append(poly)
                new_geo_data_frame["CLASSE"].append(categ)

    gdf1 = gpd.GeoDataFrame.from_dict(new_geo_data_frame, geometry="geometry", crs=crs)

    return gdf1

In [152]:
def get_model(config_file, checkpoint_file, device):
    # build the model from a config file and a checkpoint file
    model = init_model(config_file, checkpoint_file, device=device)
    return model

In [153]:
def get_paths(path):
    filenames = [f for f in os.listdir(path) if f.endswith('.tif')]
    tif_path = os.path.join(path, filenames[0]) if len(filenames) > 0 else None

    filenames = [f for f in os.listdir(path) if f.endswith('.shp')]
    shp_path = os.path.join(path, filenames[0]) if len(filenames) > 0 else None

    return tif_path, shp_path

In [154]:
config_file = '../../modelo_mamona/segformer_mit-b0_8xb2-160k_ade20k-512x512_mamona.py'
checkpoint_file = '../../modelo_mamona/iter_20000.pth'

model = get_model(config_file, checkpoint_file, 'cuda:0')

Loads checkpoint by local backend from path: ../../modelo_mamona/iter_20000.pth


/home/ipe-amarelo/CNN-SSWM/mmsegmentation-main/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/home/ipe-amarelo/CNN-SSWM/mmsegmentation-main/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


In [155]:
patch_size= 256
step = patch_size // 2

path_folder = '../../tifs/'
orto_files = os.listdir(path_folder)
    
cont = 1

print("Total de Ortos as serem processados: ", len(orto_files))
print("Ortos a serem processados:")

print(orto_files)

Total de Ortos as serem processados:  1
Ortos a serem processados:
['110158_SANTO ANTONIO_01']


In [156]:
print("---------------------------- Iniciando processamento ---------------------------- ")

for file in tqdm(orto_files):
    # Caminho da ortofoto
    path_orto = os.path.join(os.path.join(path_folder, file))
    filename_orto = os.path.basename(tif_path)[:-4]
    tif_path, shp_path = get_paths(path_orto)
    
    if tif_path is None:
        print(f'Nenhum arquivo .tif encontrado em {tif_path}')
        continue

    if shp_path is None:
        print(f'Nenhum arquivo .shp encontrado em {shp_path}')
        continue

    

    break

---------------------------- Iniciando processamento ---------------------------- 


  0%|          | 0/1 [00:00<?, ?it/s]


In [157]:
def get_img(geo_data_frame, func_latlon_xy, index, min_size):
    geo_data_frame = geo_data_frame.explode(ignore_index=True)

    bounds = geo_data_frame.iloc[[index]].bounds
    min_lat = bounds["minx"].min()
    max_lon = bounds["miny"].min()
    max_lat = bounds["maxx"].max()
    min_lon = bounds["maxy"].max()

    min_x, min_y = func_latlon_xy(min_lat, min_lon)
    max_x, max_y = func_latlon_xy(max_lat, max_lon)

    # assert max_x-min_x > 0 and max_y-min_y > 0, 'Erro de dimensoes'
    if not ((max_x - min_x > 0) and (max_y - min_y > 0)):
        print("Talhão vazio!")
        return None

    if max_x - min_x < min_size:
        max_x = min_x + min_size
    if max_y - min_y < min_size:
        max_y = min_y + min_size

    min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
    width, height = int(max_x - min_x), int(max_y - min_y)

    # Criar uma imagem em branco usando OpenCV
    image = np.zeros((width, height), dtype=np.uint8)

    geometry = geo_data_frame.geometry.iloc[index]


    pts = np.array(
        [
            func_latlon_xy(point[0], point[1])
            for point in geometry.exterior.coords[:]
        ],
        np.int32,
    )[:, ::-1]
    pts = pts - np.array([min_y, min_x])
            
    cv2.fillPoly(image, [pts], 1)
    for interior in geometry.interiors:
        pts = np.array(
            [
                func_latlon_xy(point[0], point[1])
                for point in interior.coords[:]
            ],
            np.int32,
        )[:, ::-1]
        pts = pts - np.array([min_y, min_x])
        cv2.fillPoly(image, [pts], 0)

    return (
        image,
        (min_x, min_y, max_x, max_y),
        (min_lat, max_lat, min_lon, max_lon),
    )

In [158]:
def get_image_patch(dataset, x, y, w, h):
    nc = 3
    img = np.zeros((h, w, nc), dtype=np.uint8)
    
    for i in range(nc):
        band = dataset.read(i+1, window=Window(y, x, w, h))
        nw, nh = band.shape[0], band.shape[1]
        if nw == 0 or nh == 0:
            return None
        img[0:nw, 0:nh, i] = band
    return img

In [159]:
gpd_talhoes = gpd.read_file(shp_path)
dataset = rasterio.open(tif_path)
gpd_talhoes = gpd_talhoes.to_crs(dataset.crs)
shp_all_talhoes = []

t = 0

In [160]:
mask, (min_x, min_y, max_x, max_y), (min_lat, max_lat, min_lon, max_lon) = get_img(gpd_talhoes, dataset.index, index=t, min_size=256)
min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
width, height = int(max_x-min_x), int(max_y-min_y)

In [162]:
batch_size = 32
imgs = []
positions = []
results_probs = np.zeros((2, width, height))
cont = 0

for x in tqdm(range(min_x, max_x-1, step)):
    discard_x1, discard_x2 = 0.1, 0.9
    if x == min_x:
        discard_x1 = 0.
    if x+patch_size >= max_x:
        x = max_x-patch_size
        discard_x2 = 1.
    for y in range(min_y, max_y-1, step):
        discard_y1, discard_y2 = 0.1, 0.9
        if y+patch_size >= max_y:
            discard_y2 = 1.
            y = max_y-patch_size
        if y == min_y:
            discard_y1 = 0.

        x1 = (x-min_x)+int(patch_size*discard_x1)
        y1 = (y-min_y)+int(patch_size*discard_y1)
        x2 = (x-min_x)+int(patch_size*discard_x2)
        y2 = (y-min_y)+int(patch_size*discard_y2)

        mask_patch = mask[x1:x2, y1:y2]
        if np.sum(mask_patch) == 0:
            continue

        img = get_image_patch(dataset, x, y, patch_size, patch_size)
        if img is None:
            continue

        img = img[:, :, [2, 1, 0]]
        imgs.append(img)
        positions.append([x1, x2, y1, y2, discard_x1, discard_x2, discard_y1, discard_y2, x, y])

        if len(imgs) >= batch_size:
            results_all = inference_model(model, imgs)

            for i in range(len(results_all)):
                #Image.fromarray(imgs[i]).save(f'img_{cont}.png')
                x1, x2, y1, y2, discard_x1, discard_x2, discard_y1, discard_y2, x, y = positions[i]

                patch_daninha = F.softmax(results_all[i].seg_logits.data, dim=0).cpu().numpy()

                #Image.fromarray(np.argmax(patch_daninha, axis=0).astype(np.uint8)*255).save(f'pred_{cont}.png')
                cont += 1

                patch_daninha = patch_daninha[:, int(patch_size*discard_x1):int(patch_size*discard_x2), int(patch_size*discard_y1):int(patch_size*discard_y2)]
                results_probs[:, x1:x2, y1:y2] = results_probs[:, x1:x2, y1:y2] + patch_daninha
            imgs = []
            positions = []

if len(imgs) > 0:
    results_all = inference_model(model, imgs)

    for i in range(len(results_all)):
        #Image.fromarray(imgs[i]).save(f'img_{i}.png')
        x1, x2, y1, y2, discard_x1, discard_x2, discard_y1, discard_y2, x, y = positions[i]

        patch_daninha = F.softmax(results_all[i].seg_logits.data, dim=0).cpu().numpy()
        patch_daninha = patch_daninha[:, int(patch_size*discard_x1):int(patch_size*discard_x2), int(patch_size*discard_y1):int(patch_size*discard_y2)]

        results_probs[:, x1:x2, y1:y2] = results_probs[:, x1:x2, y1:y2] + patch_daninha

results_daninha = np.argmax(results_probs, axis=0).astype(np.uint8)   
if mask is not None:
    results_daninha[mask == 0] = 0

prediction_shp = polygons_from_binary_image(results_daninha, dataset.transform, dataset.crs, min_x=min_x, min_y=min_y)
prediction_shp.to_file(os.path.join(path_orto, f'./mamona_{filename_orto}.shp'))

100%|██████████| 172/172 [01:29<00:00,  1.92it/s]
261it [00:00, 17476.55it/s]


0it [00:00, ?it/s]

462it [00:00, 4840.57it/s]
